In [4]:

!cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf1/setup.py .
!python -m pip install .

Could not make proto path relative: object_detection/protos/*.proto: No such file or directory
cp: cannot stat 'object_detection/packages/tf1/setup.py': No such file or directory
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [1]:
!pip install tensorflow==1.15
!pip install protobuf==3.20


In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.15.0'

In [11]:
!conda list

# packages in environment at /home/ratnakar/anaconda3/envs/tf1:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             5.1                       1_gnu  
absl-py                   1.4.0                    pypi_0    pypi
anyio                     3.5.0            py37h06a4308_0  
apache-beam               2.46.0                   pypi_0    pypi
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0           py37h7f8727e_0  
astor                     0.8.1                    pypi_0    pypi
astunparse                1.6.3                    pypi_0    pypi
attrs                     22.1.0           py37h06a4308_0  
avro-python3              1.10.2                   pypi_0    pypi
babel                     2.11.0           py37h06a4308_0  
backcall                  0.2.0              pyhd3eb1b0_0  
beautifulsoup4            4.11.1           py37h06a4308_0

In [2]:
!pip install  protobuf==3.20


In [ ]:
!pip install protobuf-compiler

  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Preparing metadata (setup.py) ... done
     ━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/10.6 MB 117.9 kB/s eta 0:01:21:12

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils

from object_detection.utils import config_util
from object_detection.builders import model_builder
import xml.etree.ElementTree as ET  




Using TensorFlow backend.


In [2]:


def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

     


In [9]:
chosen_model = 'ssd-mobilenet-v1-quantized'

MODELS_CONFIG = {
    'ssd-mobilenet-v1-quantized': {
        'model_name': 'ssd_mobilenet_v1_quantized_coco',
        'base_pipeline_file': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz',
    },
    'ssd-mobilenet-v2-quantized': {
        'model_name': 'ssd_mobilenet_v2_quantized_coco',
        'base_pipeline_file': 'ssd_mobilenet_v2_quantized_300x300_coco.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz',
    }
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [10]:
# Download pretrained model file from TensorFlow Model Zoo
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download base training configuration file
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/samples/configs/' + base_pipeline_file
!wget {download_config}


--2024-04-09 15:30:09--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.77.187, 142.250.195.123, 142.250.195.251, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.77.187|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85682182 (82M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz’

ssd_mobilenet_v1_qu 100%[===================>]  81.71M  10.9MB/s    in 9.8s    

2024-04-09 15:30:20 (8.38 MB/s) - ‘ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz’ saved [85682182/85682182]

--2024-04-09 15:30:21--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/samples/configs/ssd_mobilenet_v1_quantized_300x300_coco14_sync.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133,

In [1]:
pwd

'/home/ratnakar/Mobilenet'

In [2]:
import os 

In [3]:
os.chdir('models/research')

In [4]:
pwd

'/home/ratnakar/Mobilenet/models/research'

In [5]:
!python3 ./object_detection/model_main.py \
    --alsologtostderr \
    --pipeline_config_path=/home/ratnakar/Mobilenet/file.config \
    --num_train_steps=1000 \
    --model_dir=/home/ratnakar/Mobilenet/nn

Using TensorFlow backend.
W0418 11:37:47.484500 133853518783552 model_lib.py:839] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 1000
I0418 11:37:47.484661 133853518783552 config_util.py:552] Maybe overwriting train_steps: 1000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0418 11:37:47.484720 133853518783552 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0418 11:37:47.484764 133853518783552 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0418 11:37:47.484813 133853518783552 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0418 11:37:47.484870 133853518783552 model_lib.py:855] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: 

In [4]:
pwd


'/home/ratnakar/Mobilenet/models/research'

In [4]:
!python3 ./object_detection/model_main.py \
    --alsologtostderr \
    --pipeline_config_path=/home/ratnakar/Mobilenet/file.config \
    --num_train_steps=500 \
    --model_dir=/home/ratnakar/Mobilenet/ckpp

Using TensorFlow backend.
W0417 16:20:58.243543 137262248014912 model_lib.py:839] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 500
I0417 16:20:58.243706 137262248014912 config_util.py:552] Maybe overwriting train_steps: 500
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0417 16:20:58.243773 137262248014912 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0417 16:20:58.243822 137262248014912 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0417 16:20:58.243870 137262248014912 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0417 16:20:58.243934 137262248014912 model_lib.py:855] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: us

In [12]:
!python3 ./object_detection/model_main.py \
    --alsologtostderr \
    --pipeline_config_path=/home/ratnakar/Mobilenet/file.config \
    --num_train_steps=2000 \
    --model_dir=/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18

Using TensorFlow backend.
W0410 12:03:21.545356 129652689433664 model_lib.py:839] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 2000
I0410 12:03:21.545466 129652689433664 config_util.py:552] Maybe overwriting train_steps: 2000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0410 12:03:21.545526 129652689433664 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0410 12:03:21.545570 129652689433664 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0410 12:03:21.545614 129652689433664 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0410 12:03:21.545670 129652689433664 model_lib.py:855] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: 

In [7]:
pwd

'/home/ratnakar/Mobilenet'

In [12]:
!python3 ./object_detection/model_main.py \
    --alsologtostderr \
    --pipeline_config_path=/home/ratnakar/Mobilenet/file.config \
    --num_train_steps=10000 \
    --model_dir=/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18

Using TensorFlow backend.
W0410 15:21:48.210067 123984788481088 model_lib.py:839] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 10000
I0410 15:21:48.210304 123984788481088 config_util.py:552] Maybe overwriting train_steps: 10000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0410 15:21:48.210398 123984788481088 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0410 15:21:48.210488 123984788481088 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0410 15:21:48.210561 123984788481088 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0410 15:21:48.210662 123984788481088 model_lib.py:855] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs

****Train The Model****


In [9]:
!python object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path="/home/ratnakar/Mobilenet/file.config" \
    --trained_checkpoint_prefix="/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/model.ckpt" \
    --output_directory="/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18tflite" \
    --add_postprocessing_op=true

Using TensorFlow backend.
Instructions for updating:
Please use `layer.__call__` method instead.
W0417 13:29:59.375308 131379761108032 deprecation.py:323] From /home/ratnakar/anaconda3/envs/mbnet/lib/python3.7/site-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
I0417 13:30:00.258638 131379761108032 ssd_meta_arch.py:584] feature_map_spatial_dims: [(19, 19), (10, 10), (5, 5), (3, 3), (2, 2), (1, 1)]
INFO:tensorflow:depth of additional conv before box predictor: 0
I0417 13:30:00.794889 131379761108032 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0417 13:30:00.816490 131379761108032 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv be

In [1]:
pwd

'/home/ratnakar/Mobilenet'

In [4]:
import os 
os.chdir('/home/ratnakar/Mobilenet/models/research')

In [5]:
pwd


'/home/ratnakar/Mobilenet/models/research'

In [45]:
%load_ext tensorboard

%tensorboard --logdir /content/data/train_ssdlite_mobilenet_v3_large_320x320_pet

**Export model**

In [67]:
!python object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/pipeline.config \
    --trained_checkpoint_prefix=/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/model.ckpt-3398.meta \
    --output_directory=/home/ratnakar/Mobilenet/ssdlite_mobilenet_v3_large_320x320_pet


Using TensorFlow backend.
Instructions for updating:
Please use `layer.__call__` method instead.
W0410 16:22:10.175202 133613860328512 deprecation.py:323] From /home/ratnakar/anaconda3/envs/mbnet/lib/python3.7/site-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
I0410 16:22:10.787367 133613860328512 ssd_meta_arch.py:584] feature_map_spatial_dims: [(19, 19), (10, 10), (5, 5), (3, 3), (2, 2), (1, 1)]
INFO:tensorflow:depth of additional conv before box predictor: 0
I0410 16:22:11.093704 133613860328512 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0410 16:22:11.119452 133613860328512 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv be

***Export TF-Lite model***

In [68]:
!tflite_convert \
    --enable_v1_converter \
    --graph_def_file="/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18tflite/tflite_graph.pb" \
    --output_file="/home/ratnakar/Mobilenet/ssdlite_mobilenet_v3_large_320x320_pet/tflite/ssdlite_mobilenet_v3_large_320x320_pet.tflite" \
    --input_shapes=1,320,320,3 \
    --input_arrays=normalized_input_image_tensor \
    --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
    --inference_type=FLOAT \
    --allow_custom_ops


2024-04-10 16:22:34.310806: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2024-04-10 16:22:34.314813: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-04-10 16:22:34.314830: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ratnakar-ASUS-TUF-Gaming-F15-FX506LHB-FX506LHB
2024-04-10 16:22:34.314835: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ratnakar-ASUS-TUF-Gaming-F15-FX506LHB-FX506LHB
2024-04-10 16:22:34.314878: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.161.7
2024-04-10 16:22:34.314896: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.161.7
2024-04-10 16:22:34.314902: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 535.161.7
2024-04-10

In [69]:
!tflite_convert \
    --enable_v1_converter \
    --graph_def_file="/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18tflite/tflite_graph.pb" \
    --output_file="/home/ratnakar/Mobilenet/ssdlite_mobilenet_v3_large_320x320_pet" \
    --input_shapes=1,320,320,3 \
    --input_arrays=normalized_input_image_tensor \
    --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
    --inference_type=FLOAT \
    --allow_custom_ops
     

2024-04-10 16:25:35.914022: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2024-04-10 16:25:35.919234: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-04-10 16:25:35.919264: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ratnakar-ASUS-TUF-Gaming-F15-FX506LHB-FX506LHB
2024-04-10 16:25:35.919295: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ratnakar-ASUS-TUF-Gaming-F15-FX506LHB-FX506LHB
2024-04-10 16:25:35.919398: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.161.7
2024-04-10 16:25:35.919427: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.161.7
2024-04-10 16:25:35.919436: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 535.161.7
2024-04-10

****Exporting TF Lite Model****

In [71]:
!python object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path="/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/pipeline.config" \
    --trained_checkpoint_prefix="/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/model.ckpt-3398" \
    --output_directory="/home/ratnakar/Mobilenet/nn" \
    --add_postprocessing_op=true

Using TensorFlow backend.
Instructions for updating:
Please use `layer.__call__` method instead.
W0410 16:29:44.598242 136081664820288 deprecation.py:323] From /home/ratnakar/anaconda3/envs/mbnet/lib/python3.7/site-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
I0410 16:29:45.658889 136081664820288 ssd_meta_arch.py:584] feature_map_spatial_dims: [(19, 19), (10, 10), (5, 5), (3, 3), (2, 2), (1, 1)]
INFO:tensorflow:depth of additional conv before box predictor: 0
I0410 16:29:46.126297 136081664820288 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0410 16:29:46.155070 136081664820288 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv be

In [9]:
#Another saved file of tflite in pet folder 

In [8]:
!python object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path="/home/ratnakar/Mobilenet/file.config" \
    --trained_checkpoint_prefix="/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/model.ckpt-3398" \
    --output_directory="/home/ratnakar/Mobilenet/ssdlite_mobilenet_v3_large_320x320_pet/tflite" \
    --add_postprocessing_op=true
     

Using TensorFlow backend.
Instructions for updating:
Please use `layer.__call__` method instead.
W0415 17:07:29.139635 123965407040576 deprecation.py:323] From /home/ratnakar/anaconda3/envs/mbnet/lib/python3.7/site-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
I0415 17:07:29.740784 123965407040576 ssd_meta_arch.py:584] feature_map_spatial_dims: [(19, 19), (10, 10), (5, 5), (3, 3), (2, 2), (1, 1)]
INFO:tensorflow:depth of additional conv before box predictor: 0
I0415 17:07:30.053140 123965407040576 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0415 17:07:30.074427 123965407040576 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv be

In [6]:
!python object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path="/home/ratnakar/Mobilenet/file.config" \
    --trained_checkpoint_prefix="/home/ratnakar/Mobilenet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/model.ckpt-3398" \
    --output_directory="/home/ratnakar/Mobilenet/nn" \
    --add_postprocessing_op=true

Using TensorFlow backend.
Instructions for updating:
Please use `layer.__call__` method instead.
W0415 16:57:53.531757 131174693561408 deprecation.py:323] From /home/ratnakar/anaconda3/envs/mbnet/lib/python3.7/site-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
I0415 16:57:54.134557 131174693561408 ssd_meta_arch.py:584] feature_map_spatial_dims: [(19, 19), (10, 10), (5, 5), (3, 3), (2, 2), (1, 1)]
INFO:tensorflow:depth of additional conv before box predictor: 0
I0415 16:57:54.463375 131174693561408 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0415 16:57:54.481758 131174693561408 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv be